In [48]:
import requests
import bs4
from bs4 import BeautifulSoup
import re
import math
import csv
from csv import writer
import concurrent.futures
import random
from datetime import datetime
import pandas as pd
import tensorflow as tf
import os
import pandas as pd 
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense, InputLayer, Dropout, BatchNormalization
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import accuracy_score

In [2]:
#webscrape to find working proxies

#scrape from freeproxy.world
url = "https://www.freeproxy.world/"
proxylist =[]
#filter through all pages
for i in range(1,75):
    print(i)
    try:
        #filter to http proxy
        querystring = {"type":"http","anonymity":"","country":"","speed":"","port":"","page":f"{i}"}

        payload = ""
        headers = {"User-Agent": "insomnia/8.4.5"}

        #site request
        site = requests.request("GET", url, data=payload, headers=headers, params=querystring)
        soup = BeautifulSoup(site.content, 'html.parser')
        trs = soup.find_all('tr')
        for tr in trs:
            try:
                ip = tr.find('td',class_=re.compile('show-ip-div')).text.strip()
                port = tr.find('a').text.strip()
                proxylist.append(f'{ip}:{port}')
            except:
                pass
    except:
        pass
print(len(proxylist))




#scrape free proxies from proxyscrape.com
headers = {"User-Agent": "insomnia/8.4.5"}
#site request
url = 'https://proxyscrape.com/free-proxy-list'
site = requests.get(url,headers=headers)
soup = BeautifulSoup(site.content, 'html.parser')
#scrape http proxy download link
divs = soup.find_all('div', class_=re.compile('itemcard downloadcard'))
for div in divs:
    h2 = div.find('h2')
    try:
        if(h2.text.strip()=='HTTP Proxies'):
            a = div.find('a')
            download = a['download_url']
    except:
        pass

#get content of download link
try:
    site = requests.get(download,headers=headers)
    soup = BeautifulSoup(site.content, 'html.parser')
    #add proxies to testlist
    tempproxylist = soup.text.strip().split('\r')
    for proxy in tempproxylist:
        proxylist.append(proxy)

    print(len(proxylist))
except:
    pass    
#scrape proxies from free-proxy-list.net
url = 'https://free-proxy-list.net/'
#site + soup
site = requests.get(url, headers=headers)
soup = BeautifulSoup(site.content, 'html.parser')
#scrape table
try:
    table = soup.find('div', class_=re.compile('table-responsive'))
    trlabels = table.find_all('tr')
except:
    pass
#scrape proxies and add to test list
try:
    for i in trlabels:
            tdlabels = i.find_all('td')
            if(tdlabels[6].text.strip()=='yes'):
                proxylist.append(tdlabels[0].text.strip())
except:
    pass
print(len(proxylist))
#scrape proxies from hidemy.io
url = 'https://hidemy.io/en/proxy-list/'
#site request + soup
site = requests.get(url, headers=headers)
soup = BeautifulSoup(site.content, 'html.parser')
table = soup.find('div',class_=re.compile('table_block'))
#scrape table
trlabels = table.find_all('tr')
#scrape proxies and add to test list
for i in trlabels:
    tdlabels = i.find_all('td')
    try:

        if(tdlabels[4].text.strip()=='HTTP'):
            proxylist.append(tdlabels[0].text.strip())
    except:
        pass
print(len(proxylist))



#test proxies
workingProxies = []
def extract(proxy):
    works = False
    try:
        r = requests.get('https://www.whatismybrowser.com/detect/what-is-my-ip-address', proxies={'http':proxy, 'https':proxy}, timeout=2)
        soup = BeautifulSoup(r.content, 'html.parser')
        div = soup.find('div', class_='detected_result')
        ip = div.find('div').text.strip()
        print(f'{r}, {ip}')
        works = True
        workingProxies.append(proxy)
        with open('working_proxies.csv', 'w', encoding='UTF8', newline='') as workingProx:
            writer = csv.writer(workingProx)
            for k in workingProxies:
                writer.writerow([k])

    except:
        print(proxy)
    return works

#excecute test faster
with concurrent.futures.ThreadPoolExecutor() as executor:
    executor.map(extract, list(set(proxylist)))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
3700
3700
3754
173.245.49.87:80
104.21.71.88:80
31.43.179.133:8045.131.6.231:80
45.131.5.66:80
172.67.181.190:80
172.67.172.166:80

181.214.1.84:80
45.12.30.6:80
172.67.190.29:80
188.114.99.153:80
159.112.235.61:80
141.101.120.136:80
45.131.5.29:80
172.67.70.156:80
45.131.7.254:80
185.162.229.220:80159.112.235.74:80

45.131.208.113:80
172.67.254.250:80
185.238.228.128:80
45.131.4.165:80
172.67.0.66:80
45.12.31.0:80
185.162.228.230:80
172.67.25.90:80
172.67.0.2:80
185.170.166.58:80
172.64.146.92:80
172.67.245.159:80
5.182.34.95:80
188.114.96.47:80
172.67.181.91:80
104.20.251.15:80
63.141.128.3:80
185.162.228.38:80
45.12.31.117:80
172.67.72.165:80
63.141.128.104:80
45.131.4.247:80
188.114.96.10:80
34.111.135.19:80
45.12.31.171:80
141.101.120.209:80
188.114.99.92:80
172.67.181.14

In [3]:
df = pd.read_csv('NBAdata2007-2024.csv')
df

,date,homeTeam,homeTeam_id,homeTeam_points_total,homeTeam_points_q1,homeTeam_points_q2,homeTeam_points_q3,homeTeam_points_q4,homeTeam_points_1OT,homeTeam_points_2OT,...,awayTeam_3P,awayTeam_O_Reb,awayTeam_D_Reb,awayTeam_Total_Reb,awayTeam_Ast,awayTeam_Stl,awayTeam_Blk,awayTeam_TO,awayTeam_PF,winner
0,"June 17, 2024",Boston Celtics,BOS,106,28,39,19,20,0,0,...,11-37,7,28,35,18,4,4,13,20,Boston Celtics
1,"June 14, 2024",Dallas Mavericks,DAL,122,34,27,31,30,0,0,...,14-41,4,27,31,18,2,5,13,19,Dallas Mavericks
2,"June 12, 2024",Dallas Mavericks,DAL,99,31,20,19,29,0,0,...,17-46,6,30,36,26,4,6,9,19,Boston Celtics
3,"June 09, 2024",Boston Celtics,BOS,105,25,29,29,22,0,0,...,6-26,9,34,43,21,5,3,15,17,Boston Celtics
4,"June 06, 2024",Boston Celtics,BOS,107,37,26,23,21,0,0,...,7-27,10,33,43,9,8,1,11,16,Boston Celtics
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21872,"April 18, 2007",Sacramento Kings,SAC,106,30,27,23,26,0,0,...,2-14,11,36,47,19,6,6,14,26,Los Angeles Lakers
21873,"April 18, 2007",Portland Trail Blazers,POR,98,26,20,31,21,0,0,...,11-29,11,33,44,37,11,1,14,21,Golden State Warriors
21874,"April 18, 2007",Orlando Magic,ORL,94,22,17,26,29,0,0,...,4-13,5,26,31,16,6,5,15,23,Orlando Magic
21875,"April 18, 2007",New Jersey Nets,NJN,106,31,21,19,35,0,0,...,7-19,12,31,43,19,5,2,16,33,New Jersey Nets


In [4]:
#reformat the dates so they can be compared
df['date'] = df['date'].apply(lambda x: datetime.strptime(x, "%B %d, %Y"))
df

,date,homeTeam,homeTeam_id,homeTeam_points_total,homeTeam_points_q1,homeTeam_points_q2,homeTeam_points_q3,homeTeam_points_q4,homeTeam_points_1OT,homeTeam_points_2OT,...,awayTeam_3P,awayTeam_O_Reb,awayTeam_D_Reb,awayTeam_Total_Reb,awayTeam_Ast,awayTeam_Stl,awayTeam_Blk,awayTeam_TO,awayTeam_PF,winner
0,2024-06-17,Boston Celtics,BOS,106,28,39,19,20,0,0,...,11-37,7,28,35,18,4,4,13,20,Boston Celtics
1,2024-06-14,Dallas Mavericks,DAL,122,34,27,31,30,0,0,...,14-41,4,27,31,18,2,5,13,19,Dallas Mavericks
2,2024-06-12,Dallas Mavericks,DAL,99,31,20,19,29,0,0,...,17-46,6,30,36,26,4,6,9,19,Boston Celtics
3,2024-06-09,Boston Celtics,BOS,105,25,29,29,22,0,0,...,6-26,9,34,43,21,5,3,15,17,Boston Celtics
4,2024-06-06,Boston Celtics,BOS,107,37,26,23,21,0,0,...,7-27,10,33,43,9,8,1,11,16,Boston Celtics
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21872,2007-04-18,Sacramento Kings,SAC,106,30,27,23,26,0,0,...,2-14,11,36,47,19,6,6,14,26,Los Angeles Lakers
21873,2007-04-18,Portland Trail Blazers,POR,98,26,20,31,21,0,0,...,11-29,11,33,44,37,11,1,14,21,Golden State Warriors
21874,2007-04-18,Orlando Magic,ORL,94,22,17,26,29,0,0,...,4-13,5,26,31,16,6,5,15,23,Orlando Magic
21875,2007-04-18,New Jersey Nets,NJN,106,31,21,19,35,0,0,...,7-19,12,31,43,19,5,2,16,33,New Jersey Nets


In [5]:
#create list of proxies that have response code 200 when tested
proxylist =[]
with open('working_proxies.csv', 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        proxylist.append(row[0])
print(f'Proxies: {len(proxylist)}')

Proxies: 38


In [6]:
dateToday = datetime.now().date()
print(dateToday)

2024-12-15


In [7]:
#convert month to string as this is how it is presented on the website
currentMonth = dateToday.strftime("%B")
print(currentMonth)

December


In [8]:
#initalize variable holding the current year so the scraper can handle new seasons
currentYear = dateToday.year
print(currentYear)

2024


In [9]:
#create soup from current season
url = f'https://www.landofbasketball.com/results/{currentYear}_{currentYear+1}_scores.htm'
payload = ""
headers = {
    "User-Agent":'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'

}
proxy = random.choice(proxylist)
response = requests.request("GET", url, data=payload, headers=headers, proxies={'http': f"http://{proxy}, 'https:'https/{proxy}"})

print(response)
#soup
soup = BeautifulSoup(response.content, 'html.parser')

<Response [200]>


In [10]:
#extract date of most recent game included in db
mostRecentDateDB = df.iloc[0]['date']
mostRecentDateDB = mostRecentDateDB.date()
print(mostRecentDateDB)


2024-06-17


In [11]:
#handler for most reason season
if mostRecentDateDB.year <= dateToday.year and mostRecentDateDB.month < 8 and dateToday.month > 8:
    season = f'{mostRecentDateDB.year}_{mostRecentDateDB.year+1}'
    url = f'https://www.landofbasketball.com/results/{season}_scores.htm'
    payload = ""
    headers = {
        "User-Agent":'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'

    }
    proxy = random.choice(proxylist)
    response = requests.request("GET", url, data=payload, headers=headers, proxies={'http': f"http://{proxy}, 'https:'https/{proxy}"})
    #soup
    soup = BeautifulSoup(response.content, 'html.parser')
    links = soup.find_all('div', class_=re.compile('left margen-b8'))
    month = links[0].find('a').text.strip()
else:
    season = f'{dateToday.year}_{dateToday.year+1}'
    url = f'https://www.landofbasketball.com/results/{season}_scores.htm'
    month = dateToday.strftime('%B')


In [12]:
#scrape all nba games played since last update
url = f'https://www.landofbasketball.com/results/{season}_scores.htm'
payload = ""
headers = {
    "User-Agent":'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'

}
proxy = random.choice(proxylist)
response = requests.request("GET", url, data=payload, headers=headers, proxies={'http': f"http://{proxy}, 'https:'https/{proxy}"})
#soup
soup = BeautifulSoup(response.content, 'html.parser')

#get the range of months needed to scrape
month_list = []
monthTopBound = dateToday.month+1
if monthTopBound > 12:
    monthTopBound =1

if monthTopBound >= datetime.strptime(month, '%B').month:
    month_list = list(range(datetime.strptime(month, '%B').month, monthTopBound + 1))
else:
    month_list = list(range(datetime.strptime(month, '%B').month, 13)) + list(range(1, monthTopBound + 1))

month_names = [datetime(2024, m, 1).strftime('%B') for m in month_list]


links = soup.find_all('div', class_=re.compile('left margen-b8'))
selectedLinks = []
for i in range(len(links)):
    addMonth = str(links[i].find('a').text.strip())
    if addMonth in month_names:
        selectedLinks.append(links[i])

monthLinks = []
for i in range(len(selectedLinks)):
    a = selectedLinks[i].find('a')
    part = a.get('href')
    monthLinks.append(part)
print(monthLinks)

['2024_2025_oct_scores.htm', '2024_2025_nov_scores.htm', '2024_2025_dec_scores.htm', '2024_2025_jan_scores.htm']


In [13]:
#includes games that haven't been played yet, but those are necessary for inputs
gameLinks = []
for part in monthLinks:    
    url = f'https://www.landofbasketball.com/results/{part}'
    payload = ""
    headers = {
        "User-Agent":'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'

    }
    proxy = random.choice(proxylist)
    response = requests.request("GET", url, data=payload, headers=headers, proxies={'http': f"http://{proxy}, 'https:'https/{proxy}"})

    soup = BeautifulSoup(response.content, 'html.parser')
    links = soup.find_all('div', class_=re.compile('clearfix clear margin-t1 margen-b2'))
    for i in range(len(links)):
        try:
            a = links[i].find('a')
            part = a.get('href')
            gameLinks.append(part)
        except:
            pass
print(f'Game Links found: {len(gameLinks)}')

Game Links found: 716


In [14]:
#initalize csv
head = ['date', 'homeTeam', 'homeTeam_id', 'homeTeam_points_total', 'homeTeam_points_q1', 'homeTeam_points_q2', 'homeTeam_points_q3', 'homeTeam_points_q4',
        'homeTeam_points_1OT', 'homeTeam_points_2OT', 'homeTeam_points_3OT', 'homeTeam_points_4OT', 
        'homeTeam_FG', 'homeTeam_3P', 'homeTeam_O_Reb', 'homeTeam_D_Reb', 'homeTeam_Total_Reb', 'homeTeam_Ast', 'homeTeam_Stl', 'homeTeam_Blk', 
        'homeTeam_TO', 'homeTeam_PF', 'awayTeam', 'awayTeam_id', 'awayTeam_points_total', 'awayTeam_points_q1', 'awayTeam_points_q2', 'awayTeam_points_q3', 
        'awayTeam_points_q4', 'awayTeam_points_1OT', 'awayTeam_points_2OT', 'awayTeam_points_3OT', 'awayTeam_points_4OT', 'awayTeam_FG', 'awayTeam_3P', 
        'awayTeam_O_Reb', 'awayTeam_D_Reb', 'awayTeam_Total_Reb', 'awayTeam_Ast', 'awayTeam_Stl', 'awayTeam_Blk', 'awayTeam_TO', 'awayTeam_PF', 'winner']

with open('updateDB.csv', 'w', encoding='UTF8', newline='') as nbaDatabase:
    writer = csv.writer(nbaDatabase)
    writer.writerow(head)

In [15]:
teamID_dict = {
    "Atlanta Hawks": "ATL",
    "Boston Celtics": "BOS",
    "Brooklyn Nets": "BKN",
    "Charlotte Hornets": "CHA",
    "Chicago Bulls": "CHI",
    "Cleveland Cavaliers": "CLE",
    "Dallas Mavericks": "DAL",
    "Denver Nuggets": "DEN",
    "Detroit Pistons": "DET",
    "Golden State Warriors": "GSW",
    "Houston Rockets": "HOU",
    "Indiana Pacers": "IND",
    "Los Angeles Clippers": "LAC",
    "Los Angeles Lakers": "LAL",
    "Memphis Grizzlies": "MEM",
    "Miami Heat": "MIA",
    "Milwaukee Bucks": "MIL",
    "Minnesota Timberwolves": "MIN",
    "New Orleans Pelicans": "NOP",
    "New York Knicks": "NYK",
    "Oklahoma City Thunder": "OKC",
    "Orlando Magic": "ORL",
    "Philadelphia 76ers": "PHI",
    "Phoenix Suns": "PHX",
    "Portland Trail Blazers": "POR",
    "Sacramento Kings": "SAC",
    "San Antonio Spurs": "SAS",
    "Toronto Raptors": "TOR",
    "Utah Jazz": "UTA",
    "Washington Wizards": "WAS",
    "Charlotte Bobcats": "CHA",
    "New Orleans Hornets": "NOP",
    "Oklahoma City SuperSonics": "OKC",
    "New Jersey Nets": "BKN",
    "Seattle SuperSonics": "OKC"
}

In [16]:
followingGames = [link for link in gameLinks if 'head_to_head' in link]
gameLinks = [link for link in gameLinks if 'head_to_head' not in link]
print(followingGames[0])
print(gameLinks[len(gameLinks)-1])

../head_to_head/pelicans_vs_pacers_all_time.htm
../box_scores/2024/1214HOUOKC.htm


In [17]:
for part in gameLinks:
    #access next game
    url = f'https://www.landofbasketball.com/{part}'
    payload = ""
    headers = {
        "User-Agent":'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'

    }
    proxy = random.choice(proxylist)
    response = requests.request("GET", url, data=payload, headers=headers, proxies={'http': f"http://{proxy}, 'https:'https/{proxy}"})

    #soup
    soup = BeautifulSoup(response.content, 'html.parser')

    #initialize variables
    date = None
    awayTeam = None
    homeTeam = None
    awayTeam_id = None
    homeTeam_id = None
    awayTeam_points_total = None
    homeTeam_points_total = None
    awayTeam_points_q1 = None
    homeTeam_points_q1 = None
    awayTeam_points_q2 = None
    homeTeam_points_q2 = None
    awayTeam_points_q3 = None
    homeTeam_points_q3 = None
    awayTeam_points_q4 = None
    homeTeam_points_q4 = None
    awayTeam_points_1OT = 0
    homeTeam_points_1OT = 0
    awayTeam_points_2OT = 0
    homeTeam_points_2OT = 0
    awayTeam_points_3OT = 0
    homeTeam_points_3OT = 0
    awayTeam_points_4OT = 0
    homeTeam_points_4OT = 0
    awayTeam_FG = None
    homeTeam_FG = None
    awayTeam_3P = None
    homeTeam_3P = None
    awayTeam_FT = None
    homeTeam_FT = None
    awayTeam_O_Reb = None
    homeTeam_O_Reb = None
    awayTeam_D_Reb = None
    homeTeam_D_reb = None
    awayTeam_Total_Reb = None
    homeTeam_Total_Reb = None
    awayTeam_Ast = None
    homeTeam_Ast = None
    awayTeam_Stl = None
    homeTeam_Stl = None
    awayTeam_Blk = None
    homeTeam_Blk = None
    awayTeam_TO = None
    homeTeam_TO = None
    awayTeam_PF = None
    homeTeam_PF = None
    overtimes = 0
    winner = None


    overtime_points = {
    'homeTeam_points_1OT': 0,
    'awayTeam_points_1OT': 0,
    'homeTeam_points_2OT': 0,
    'awayTeam_points_2OT': 0,
    'homeTeam_points_3OT': 0,
    'awayTeam_points_3OT': 0,
    'homeTeam_points_4OT': 0,
    'awayTeam_points_4OT': 0
    }   

    #scrape+clean date
    dateHome = soup.find('div', class_=re.compile('pad-b10 pad-l5'))
    dateHome = dateHome.text.strip().split('\n')
    date = dateHome[0].split(': ')
    date = date[1]

    #scrape+clean homeTeam
    homeTeam = dateHome[1].split(': ')
    homeTeam = homeTeam[1]

    table = soup.find('table', class_=re.compile('size-11 a-left'))
    tableText = table.text.strip().split('\n')
    for i in range(len(tableText)):
        newI = tableText[i].replace('\t', '')
        tableText[i] = newI

    filtered_text = [item for item in tableText if item]
    filtered_text.pop(2)
    team1 = filtered_text[:2]
    team2 = filtered_text[2:]

    #scrape+clean awayTeam and point totals
    if homeTeam in team1:
        homeTeam_points_total = team1[0]
        awayTeam = team2[1]
        awayTeam_points_total = team2[0]
    else:
        homeTeam_points_total = team2[0]
        awayTeam = team1[1]
        awayTeam_points_total = team1[0]

    if 'OT' in homeTeam_points_total:
        getOT = homeTeam_points_total.split(' ')
        try:
            overtimes = int(getOT[1].replace('(', '').replace(')', '').replace('O', '').replace('T', ''))
        except:
            overtimes = 1
    if 'OT' in awayTeam_points_total:
        getOT = awayTeam_points_total.split(' ')
        try:
            overtimes = int(getOT[1].replace('(', '').replace(')', '').replace('O', '').replace('T', ''))
        except:
            overtimes = 1

    #extract team ids from dict
    homeTeam_id = teamID_dict[f'{homeTeam}']
    awayTeam_id = teamID_dict[f'{awayTeam}']

    #scrape + clean points per quarter
    quarterTable = soup.find('table', re.compile('by pad-y5 margen-b5'))
    quarterStats = quarterTable.find_all('td')
    qStatList = []


    for stat in quarterStats:
        qStatList.append(stat.text.strip())
    team1qStats = qStatList[overtimes+6:overtimes+overtimes+12]
    team2qStats = qStatList[overtimes+overtimes+12:]
    if homeTeam_id in str(team1qStats[0]):
        homeTeam_points_q1 = team1qStats[1]
        homeTeam_points_q2 = team1qStats[2]
        homeTeam_points_q3 = team1qStats[3]
        homeTeam_points_q4 = team1qStats[4]
        homeTeam_points_total = team1qStats[len(team1qStats)-1]
        awayTeam_points_q1 = team2qStats[1]
        awayTeam_points_q2 = team2qStats[2]
        awayTeam_points_q3 = team2qStats[3]
        awayTeam_points_q4 = team2qStats[4]
        awayTeam_points_total = team2qStats[len(team2qStats)-1]
    if awayTeam_id in str(team1qStats[0]):
        homeTeam_points_q1 = team2qStats[1]
        homeTeam_points_q2 = team2qStats[2]
        homeTeam_points_q3 = team2qStats[3]
        homeTeam_points_q4 = team2qStats[4]
        homeTeam_points_total = team2qStats[len(team2qStats)-1]
        awayTeam_points_q1 = team1qStats[1]
        awayTeam_points_q2 = team1qStats[2]
        awayTeam_points_q3 = team1qStats[3]
        awayTeam_points_q4 = team1qStats[4]
        awayTeam_points_total = team1qStats[len(team1qStats)-1]


    team1qStats.pop()
    team2qStats.pop()

    try:
        team1OTStats = team1qStats[5:]
        team2OTStats = team2qStats[5:]

        for i in range(len(team1OTStats)):
            if homeTeam_id in str(team1qStats[0]):
                overtime_points[f'homeTeam_points_{i+1}OT'] = team1OTStats[i]
                overtime_points[f'awayTeam_points_{i+1}OT'] = team2OTStats[i]
            if awayTeam_id in str(team1qStats):
                overtime_points[f'homeTeam_points_{i+1}OT'] = team2OTStats[i]
                overtime_points[f'awayTeam_points_{i+1}OT'] = team1OTStats[i]
    except:
        pass

    awayTeam_points_1OT = overtime_points['awayTeam_points_1OT']
    homeTeam_points_1OT = overtime_points['homeTeam_points_1OT']
    awayTeam_points_2OT = overtime_points['awayTeam_points_2OT']
    homeTeam_points_2OT = overtime_points['homeTeam_points_2OT']
    awayTeam_points_3OT = overtime_points['awayTeam_points_3OT']
    homeTeam_points_3OT = overtime_points['homeTeam_points_3OT']
    awayTeam_points_4OT = overtime_points['awayTeam_points_4OT']
    homeTeam_points_4OT = overtime_points['awayTeam_points_4OT']

    #scrape + clean all other stats
    statTables = soup.find_all('div', class_=re.compile('clearfix clear pad-y5'))
    for i in range(len(statTables)):
        team1 = statTables[i].find('h3', class_=re.compile('d-inline-block a-middle size-11 margen-t0 margen-b0 negri'))
        if awayTeam in team1.text.strip():
            awayTeamStats = statTables[i].find('tr', class_=re.compile('color-5 negri a-top'))
            awayTeamStats = awayTeamStats.text.strip().split('\n')
            awayTeam_FG = awayTeamStats[3]
            awayTeam_3P = awayTeamStats[4]
            awayTeam_FT = awayTeamStats[5]
            awayTeam_O_Reb = awayTeamStats[6]
            awayTeam_D_Reb = awayTeamStats[7]
            awayTeam_Total_Reb = awayTeamStats[8]
            awayTeam_Ast = awayTeamStats[9]
            awayTeam_Stl = awayTeamStats[10]
            awayTeam_Blk = awayTeamStats[11]
            awayTeam_TO = awayTeamStats[12]
            awayTeam_PF = awayTeamStats[13]
        if homeTeam in team1.text.strip():
            homeTeamStats = statTables[i].find('tr', class_=re.compile('color-5 negri a-top'))
            homeTeamStats = homeTeamStats.text.strip().split('\n')
            homeTeam_FG = homeTeamStats[3]
            homeTeam_3P = homeTeamStats[4]
            homeTeam_FT = homeTeamStats[5]
            homeTeam_O_Reb = homeTeamStats[6]
            homeTeam_D_Reb = homeTeamStats[7]
            homeTeam_Total_Reb = homeTeamStats[8]
            homeTeam_Ast = homeTeamStats[9]
            homeTeam_Stl = homeTeamStats[10]
            homeTeam_Blk = homeTeamStats[11]
            homeTeam_TO = homeTeamStats[12]
            homeTeam_PF = homeTeamStats[13]

    #extract winner
    home_points = int(re.search(r'\d+', homeTeam_points_total).group())
    away_points = int(re.search(r'\d+', awayTeam_points_total).group())
    if home_points > away_points:
        winner = homeTeam
    elif away_points > home_points:
        winner = awayTeam
    else:
        winner = "Draw"



    print(f'Scraping {homeTeam} vs {awayTeam}: {date}...')
    #write game to csv

    gameStats = [
        [date, homeTeam, homeTeam_id, homeTeam_points_total, homeTeam_points_q1, homeTeam_points_q2, homeTeam_points_q3, homeTeam_points_q4, homeTeam_points_1OT, homeTeam_points_2OT, homeTeam_points_3OT, homeTeam_points_4OT,
        homeTeam_FG, homeTeam_3P, homeTeam_O_Reb, homeTeam_D_Reb, homeTeam_Total_Reb, homeTeam_Ast, homeTeam_Stl, homeTeam_Blk, 
        homeTeam_TO, homeTeam_PF, awayTeam, awayTeam_id, awayTeam_points_total, awayTeam_points_q1, awayTeam_points_q2, awayTeam_points_q3,
        awayTeam_points_q4, awayTeam_points_1OT, awayTeam_points_2OT, awayTeam_points_3OT,
        awayTeam_points_4OT, awayTeam_FG, awayTeam_3P, awayTeam_O_Reb, awayTeam_D_Reb, awayTeam_Total_Reb, awayTeam_Ast, awayTeam_Stl, 
        awayTeam_Blk, awayTeam_TO, awayTeam_PF, winner],
    ]

    with open('updateDB.csv', 'a', encoding='UTF8', newline='') as nbaDatabase:
        writer = csv.writer(nbaDatabase)      
        writer.writerows(gameStats) 

Scraping Boston Celtics vs New York Knicks: October 22, 2024...
Scraping Los Angeles Lakers vs Minnesota Timberwolves: October 22, 2024...
Scraping Detroit Pistons vs Indiana Pacers: October 23, 2024...
Scraping Atlanta Hawks vs Brooklyn Nets: October 23, 2024...
Scraping Miami Heat vs Orlando Magic: October 23, 2024...
Scraping Philadelphia 76ers vs Milwaukee Bucks: October 23, 2024...
Scraping Toronto Raptors vs Cleveland Cavaliers: October 23, 2024...
Scraping Houston Rockets vs Charlotte Hornets: October 23, 2024...
Scraping New Orleans Pelicans vs Chicago Bulls: October 23, 2024...
Scraping Utah Jazz vs Memphis Grizzlies: October 23, 2024...
Scraping Los Angeles Clippers vs Phoenix Suns: October 23, 2024...
Scraping Portland Trail Blazers vs Golden State Warriors: October 23, 2024...
Scraping Washington Wizards vs Boston Celtics: October 24, 2024...
Scraping Dallas Mavericks vs San Antonio Spurs: October 24, 2024...
Scraping Denver Nuggets vs Oklahoma City Thunder: October 24, 202

In [30]:
inputGames = []
for part in followingGames:
    #access next game
    url = f'https://www.landofbasketball.com/{part}'
    payload = ""
    headers = {
        "User-Agent":'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'

    }
    proxy = random.choice(proxylist)
    response = requests.request("GET", url, data=payload, headers=headers, proxies={'http': f"http://{proxy}, 'https:'https/{proxy}"})

    #soup
    soup = BeautifulSoup(response.content, 'html.parser')

    #extract game header
    gameHeader = soup.find('div', class_=re.compile('rd-100-70 size-13 margen-t2 margen-b5'))

    teamNames = gameHeader.find_all('a')
    teamNames_clean = []
    for team in teamNames:
        teamNames_clean.append(team.text.strip())
    awayTeam = teamNames_clean[0]
    homeTeam = teamNames_clean[1]
    if awayTeam and homeTeam not in inputGames:
        inputGames.append(homeTeam)
        inputGames.append(awayTeam)
    else:
        break

    

In [31]:
print(inputGames)

['Indiana Pacers', 'New Orleans Pelicans', 'Washington Wizards', 'Boston Celtics', 'San Antonio Spurs', 'Minnesota Timberwolves', 'Phoenix Suns', 'Portland Trail Blazers', 'Los Angeles Lakers', 'Memphis Grizzlies', 'Orlando Magic', 'New York Knicks', 'Golden State Warriors', 'Dallas Mavericks', 'Detroit Pistons', 'Miami Heat', 'Charlotte Hornets', 'Philadelphia 76ers', 'Toronto Raptors', 'Chicago Bulls', 'Brooklyn Nets', 'Cleveland Cavaliers', 'Sacramento Kings', 'Denver Nuggets', 'Los Angeles Clippers', 'Utah Jazz', 'Oklahoma City Thunder', 'Milwaukee Bucks']


In [36]:
dfUpdate = pd.read_csv('updateDB.csv')
dfUpdate

,date,homeTeam,homeTeam_id,homeTeam_points_total,homeTeam_points_q1,homeTeam_points_q2,homeTeam_points_q3,homeTeam_points_q4,homeTeam_points_1OT,homeTeam_points_2OT,...,awayTeam_3P,awayTeam_O_Reb,awayTeam_D_Reb,awayTeam_Total_Reb,awayTeam_Ast,awayTeam_Stl,awayTeam_Blk,awayTeam_TO,awayTeam_PF,winner
0,"October 22, 2024",Boston Celtics,BOS,132,43,31,39,19,0,0,...,11-30,5,29,34,20,2,3,11,12,Boston Celtics
1,"October 22, 2024",Los Angeles Lakers,LAL,110,22,33,27,28,0,0,...,13-41,12,35,47,17,4,1,15,22,Los Angeles Lakers
2,"October 23, 2024",Detroit Pistons,DET,109,31,27,32,19,0,0,...,9-31,7,32,39,32,10,8,15,23,Indiana Pacers
3,"October 23, 2024",Atlanta Hawks,ATL,120,30,21,32,37,0,0,...,17-43,12,31,43,21,8,6,17,32,Atlanta Hawks
4,"October 23, 2024",Miami Heat,MIA,97,32,22,18,25,0,0,...,18-49,18,39,57,28,8,8,11,21,Orlando Magic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371,"December 13, 2024",Denver Nuggets,DEN,120,29,19,35,37,0,0,...,14-41,5,26,31,24,18,2,19,22,Denver Nuggets
372,"December 13, 2024",Utah Jazz,UTA,126,39,32,34,21,0,0,...,22-45,9,28,37,36,10,7,7,18,Phoenix Suns
373,"December 13, 2024",Portland Trail Blazers,POR,116,25,17,46,28,0,0,...,14-30,13,28,41,32,5,6,14,13,San Antonio Spurs
374,"December 14, 2024",Milwaukee Bucks,MIL,110,26,29,27,28,0,0,...,10-24,7,37,44,23,3,6,13,20,Milwaukee Bucks


In [37]:
#fix time format
dfUpdate['date'] = dfUpdate['date'].apply(lambda x: datetime.strptime(x, "%B %d, %Y"))
dfUpdate

,date,homeTeam,homeTeam_id,homeTeam_points_total,homeTeam_points_q1,homeTeam_points_q2,homeTeam_points_q3,homeTeam_points_q4,homeTeam_points_1OT,homeTeam_points_2OT,...,awayTeam_3P,awayTeam_O_Reb,awayTeam_D_Reb,awayTeam_Total_Reb,awayTeam_Ast,awayTeam_Stl,awayTeam_Blk,awayTeam_TO,awayTeam_PF,winner
0,2024-10-22,Boston Celtics,BOS,132,43,31,39,19,0,0,...,11-30,5,29,34,20,2,3,11,12,Boston Celtics
1,2024-10-22,Los Angeles Lakers,LAL,110,22,33,27,28,0,0,...,13-41,12,35,47,17,4,1,15,22,Los Angeles Lakers
2,2024-10-23,Detroit Pistons,DET,109,31,27,32,19,0,0,...,9-31,7,32,39,32,10,8,15,23,Indiana Pacers
3,2024-10-23,Atlanta Hawks,ATL,120,30,21,32,37,0,0,...,17-43,12,31,43,21,8,6,17,32,Atlanta Hawks
4,2024-10-23,Miami Heat,MIA,97,32,22,18,25,0,0,...,18-49,18,39,57,28,8,8,11,21,Orlando Magic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371,2024-12-13,Denver Nuggets,DEN,120,29,19,35,37,0,0,...,14-41,5,26,31,24,18,2,19,22,Denver Nuggets
372,2024-12-13,Utah Jazz,UTA,126,39,32,34,21,0,0,...,22-45,9,28,37,36,10,7,7,18,Phoenix Suns
373,2024-12-13,Portland Trail Blazers,POR,116,25,17,46,28,0,0,...,14-30,13,28,41,32,5,6,14,13,San Antonio Spurs
374,2024-12-14,Milwaukee Bucks,MIL,110,26,29,27,28,0,0,...,10-24,7,37,44,23,3,6,13,20,Milwaukee Bucks


In [38]:
dfUpdate = dfUpdate.iloc[::-1].reset_index(drop=True)
dfUpdate

,date,homeTeam,homeTeam_id,homeTeam_points_total,homeTeam_points_q1,homeTeam_points_q2,homeTeam_points_q3,homeTeam_points_q4,homeTeam_points_1OT,homeTeam_points_2OT,...,awayTeam_3P,awayTeam_O_Reb,awayTeam_D_Reb,awayTeam_Total_Reb,awayTeam_Ast,awayTeam_Stl,awayTeam_Blk,awayTeam_TO,awayTeam_PF,winner
0,2024-12-14,Oklahoma City Thunder,OKC,111,18,23,34,36,0,0,...,11-46,19,30,49,17,7,3,16,25,Oklahoma City Thunder
1,2024-12-14,Milwaukee Bucks,MIL,110,26,29,27,28,0,0,...,10-24,7,37,44,23,3,6,13,20,Milwaukee Bucks
2,2024-12-13,Portland Trail Blazers,POR,116,25,17,46,28,0,0,...,14-30,13,28,41,32,5,6,14,13,San Antonio Spurs
3,2024-12-13,Utah Jazz,UTA,126,39,32,34,21,0,0,...,22-45,9,28,37,36,10,7,7,18,Phoenix Suns
4,2024-12-13,Denver Nuggets,DEN,120,29,19,35,37,0,0,...,14-41,5,26,31,24,18,2,19,22,Denver Nuggets
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371,2024-10-23,Miami Heat,MIA,97,32,22,18,25,0,0,...,18-49,18,39,57,28,8,8,11,21,Orlando Magic
372,2024-10-23,Atlanta Hawks,ATL,120,30,21,32,37,0,0,...,17-43,12,31,43,21,8,6,17,32,Atlanta Hawks
373,2024-10-23,Detroit Pistons,DET,109,31,27,32,19,0,0,...,9-31,7,32,39,32,10,8,15,23,Indiana Pacers
374,2024-10-22,Los Angeles Lakers,LAL,110,22,33,27,28,0,0,...,13-41,12,35,47,17,4,1,15,22,Los Angeles Lakers


In [39]:
updatedDatabase = pd.concat([dfUpdate, df], ignore_index=True)
updatedDatabase

,date,homeTeam,homeTeam_id,homeTeam_points_total,homeTeam_points_q1,homeTeam_points_q2,homeTeam_points_q3,homeTeam_points_q4,homeTeam_points_1OT,homeTeam_points_2OT,...,awayTeam_3P,awayTeam_O_Reb,awayTeam_D_Reb,awayTeam_Total_Reb,awayTeam_Ast,awayTeam_Stl,awayTeam_Blk,awayTeam_TO,awayTeam_PF,winner
0,2024-12-14,Oklahoma City Thunder,OKC,111,18,23,34,36,0,0,...,11-46,19,30,49,17,7,3,16,25,Oklahoma City Thunder
1,2024-12-14,Milwaukee Bucks,MIL,110,26,29,27,28,0,0,...,10-24,7,37,44,23,3,6,13,20,Milwaukee Bucks
2,2024-12-13,Portland Trail Blazers,POR,116,25,17,46,28,0,0,...,14-30,13,28,41,32,5,6,14,13,San Antonio Spurs
3,2024-12-13,Utah Jazz,UTA,126,39,32,34,21,0,0,...,22-45,9,28,37,36,10,7,7,18,Phoenix Suns
4,2024-12-13,Denver Nuggets,DEN,120,29,19,35,37,0,0,...,14-41,5,26,31,24,18,2,19,22,Denver Nuggets
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22248,2007-04-18,Sacramento Kings,SAC,106,30,27,23,26,0,0,...,2-14,11,36,47,19,6,6,14,26,Los Angeles Lakers
22249,2007-04-18,Portland Trail Blazers,POR,98,26,20,31,21,0,0,...,11-29,11,33,44,37,11,1,14,21,Golden State Warriors
22250,2007-04-18,Orlando Magic,ORL,94,22,17,26,29,0,0,...,4-13,5,26,31,16,6,5,15,23,Orlando Magic
22251,2007-04-18,New Jersey Nets,NJN,106,31,21,19,35,0,0,...,7-19,12,31,43,19,5,2,16,33,New Jersey Nets


In [41]:
updatedDatabase.to_csv(f'NBADatabase{dateToday}', index=False)

In [44]:
dfMine = pd.read_csv(f'NBADatabase{dateToday}')

dfMine['date'] = pd.to_datetime(dfMine['date'])
# Function to calculate the season
def get_season(date):
    year = date.year
    if date.month >= 10:  # October to December belong to the current year's season
        return f"{year}-{year + 1}"
    else:  # January to June belong to the previous year's season
        return f"{year - 1}-{year}"

# Apply the season calculation to the DataFrame
dfMine['season'] = dfMine['date'].apply(get_season)

dfMine

,date,homeTeam,homeTeam_id,homeTeam_points_total,homeTeam_points_q1,homeTeam_points_q2,homeTeam_points_q3,homeTeam_points_q4,homeTeam_points_1OT,homeTeam_points_2OT,...,awayTeam_O_Reb,awayTeam_D_Reb,awayTeam_Total_Reb,awayTeam_Ast,awayTeam_Stl,awayTeam_Blk,awayTeam_TO,awayTeam_PF,winner,season
0,2024-12-14,Oklahoma City Thunder,OKC,111,18,23,34,36,0,0,...,19,30,49,17,7,3,16,25,Oklahoma City Thunder,2024-2025
1,2024-12-14,Milwaukee Bucks,MIL,110,26,29,27,28,0,0,...,7,37,44,23,3,6,13,20,Milwaukee Bucks,2024-2025
2,2024-12-13,Portland Trail Blazers,POR,116,25,17,46,28,0,0,...,13,28,41,32,5,6,14,13,San Antonio Spurs,2024-2025
3,2024-12-13,Utah Jazz,UTA,126,39,32,34,21,0,0,...,9,28,37,36,10,7,7,18,Phoenix Suns,2024-2025
4,2024-12-13,Denver Nuggets,DEN,120,29,19,35,37,0,0,...,5,26,31,24,18,2,19,22,Denver Nuggets,2024-2025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22248,2007-04-18,Sacramento Kings,SAC,106,30,27,23,26,0,0,...,11,36,47,19,6,6,14,26,Los Angeles Lakers,2006-2007
22249,2007-04-18,Portland Trail Blazers,POR,98,26,20,31,21,0,0,...,11,33,44,37,11,1,14,21,Golden State Warriors,2006-2007
22250,2007-04-18,Orlando Magic,ORL,94,22,17,26,29,0,0,...,5,26,31,16,6,5,15,23,Orlando Magic,2006-2007
22251,2007-04-18,New Jersey Nets,NJN,106,31,21,19,35,0,0,...,12,31,43,19,5,2,16,33,New Jersey Nets,2006-2007


In [45]:
#sort by season
dfMine = dfMine.sort_values(by=['season', 'date']).reset_index(drop=True)

def calculate_win_percentage_for_season(season_df):
    # Initialize team stats for the season
    team_stats = {}
    
    # Function to calculate dynamic win percentage
    def calculate_win_percentage(row):
        nonlocal team_stats

        # Extract teams
        home_team = row['homeTeam']
        away_team = row['awayTeam']
        winner = row['winner']

        # Initialize teams in the stats dictionary if not already present
        for team in [home_team, away_team]:
            if team not in team_stats:
                team_stats[team] = {'wins': 0, 'games': 0}

        # Update games played for both teams
        team_stats[home_team]['games'] += 1
        team_stats[away_team]['games'] += 1

        # Update wins for the winner
        if winner == home_team:
            team_stats[home_team]['wins'] += 1
        elif winner == away_team:
            team_stats[away_team]['wins'] += 1

        # Calculate win percentage for both teams
        home_win_pct = (team_stats[home_team]['wins'] / team_stats[home_team]['games']) * 100
        away_win_pct = (team_stats[away_team]['wins'] / team_stats[away_team]['games']) * 100

        return pd.Series([home_win_pct, away_win_pct])

    # Apply the calculation to the season's DataFrame
    season_df[['homeWinPct', 'awayWinPct']] = season_df.apply(calculate_win_percentage, axis=1)
    return season_df

# Apply the win percentage calculation for each season
dfMine = dfMine.groupby('season', group_keys=False).apply(calculate_win_percentage_for_season)

dfMine

,date,homeTeam,homeTeam_id,homeTeam_points_total,homeTeam_points_q1,homeTeam_points_q2,homeTeam_points_q3,homeTeam_points_q4,homeTeam_points_1OT,homeTeam_points_2OT,...,awayTeam_Total_Reb,awayTeam_Ast,awayTeam_Stl,awayTeam_Blk,awayTeam_TO,awayTeam_PF,winner,season,homeWinPct,awayWinPct
0,2007-04-18,Utah Jazz,UTA,101,20,30,25,26,0,0,...,25,17,8,2,11,29,Utah Jazz,2006-2007,100.000000,0.000000
1,2007-04-18,Toronto Raptors,TOR,119,28,29,29,33,0,0,...,41,22,4,5,15,25,Philadelphia 76ers,2006-2007,0.000000,100.000000
2,2007-04-18,Seattle Supersonics,SEA,75,16,21,22,16,0,0,...,38,26,7,5,9,21,Dallas Mavericks,2006-2007,0.000000,100.000000
3,2007-04-18,San Antonio Spurs,SAS,77,16,24,22,15,0,0,...,55,28,5,3,15,21,Denver Nuggets,2006-2007,0.000000,100.000000
4,2007-04-18,Sacramento Kings,SAC,106,30,27,23,26,0,0,...,47,19,6,6,14,26,Los Angeles Lakers,2006-2007,0.000000,100.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22248,2024-12-13,Minnesota Timberwolves,MIN,97,32,18,27,20,0,0,...,53,17,7,6,21,21,Minnesota Timberwolves,2024-2025,54.166667,52.000000
22249,2024-12-13,Philadelphia 76ers,PHI,107,22,28,33,24,0,0,...,41,35,9,4,10,21,Indiana Pacers,2024-2025,30.434783,42.307692
22250,2024-12-13,Cleveland Cavaliers,CLE,115,25,32,30,28,0,0,...,44,24,7,7,17,28,Cleveland Cavaliers,2024-2025,84.615385,13.043478
22251,2024-12-14,Oklahoma City Thunder,OKC,111,18,23,34,36,0,0,...,49,17,7,3,16,25,Oklahoma City Thunder,2024-2025,80.000000,65.384615


In [46]:
# Function to classify playoff games
def classify_playoff_games(season_df):
    # Get the current season
    season = season_df['season'].iloc[0]
    
    # Determine total regular-season games for the season
    if season == "2020-21":
        total_regular_season_games = 1080  # 72 games per team
    elif season == "2019-20":
        total_regular_season_games = 971  # COVID-shortened season
    else:
        total_regular_season_games = 1230  # Full regular season (82 games per team)
    
    # Add a game number column for the season
    season_df['gameNumber'] = range(1, len(season_df) + 1)
    # Classify games as playoffs if they occur after the regular season
    season_df['isPlayoffGame'] = season_df['gameNumber'] > total_regular_season_games
    return season_df

# Apply the playoff classification for each season
dfMine = dfMine.groupby('season', group_keys=False).apply(classify_playoff_games)

dfMine

,date,homeTeam,homeTeam_id,homeTeam_points_total,homeTeam_points_q1,homeTeam_points_q2,homeTeam_points_q3,homeTeam_points_q4,homeTeam_points_1OT,homeTeam_points_2OT,...,awayTeam_Stl,awayTeam_Blk,awayTeam_TO,awayTeam_PF,winner,season,homeWinPct,awayWinPct,gameNumber,isPlayoffGame
0,2007-04-18,Utah Jazz,UTA,101,20,30,25,26,0,0,...,8,2,11,29,Utah Jazz,2006-2007,100.000000,0.000000,1,False
1,2007-04-18,Toronto Raptors,TOR,119,28,29,29,33,0,0,...,4,5,15,25,Philadelphia 76ers,2006-2007,0.000000,100.000000,2,False
2,2007-04-18,Seattle Supersonics,SEA,75,16,21,22,16,0,0,...,7,5,9,21,Dallas Mavericks,2006-2007,0.000000,100.000000,3,False
3,2007-04-18,San Antonio Spurs,SAS,77,16,24,22,15,0,0,...,5,3,15,21,Denver Nuggets,2006-2007,0.000000,100.000000,4,False
4,2007-04-18,Sacramento Kings,SAC,106,30,27,23,26,0,0,...,6,6,14,26,Los Angeles Lakers,2006-2007,0.000000,100.000000,5,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22248,2024-12-13,Minnesota Timberwolves,MIN,97,32,18,27,20,0,0,...,7,6,21,21,Minnesota Timberwolves,2024-2025,54.166667,52.000000,372,False
22249,2024-12-13,Philadelphia 76ers,PHI,107,22,28,33,24,0,0,...,9,4,10,21,Indiana Pacers,2024-2025,30.434783,42.307692,373,False
22250,2024-12-13,Cleveland Cavaliers,CLE,115,25,32,30,28,0,0,...,7,7,17,28,Cleveland Cavaliers,2024-2025,84.615385,13.043478,374,False
22251,2024-12-14,Oklahoma City Thunder,OKC,111,18,23,34,36,0,0,...,7,3,16,25,Oklahoma City Thunder,2024-2025,80.000000,65.384615,375,False


In [47]:
def calculate_win_loss(season_df):
    # Initialize dictionaries to track wins and losses
    win_counts = {}
    loss_counts = {}

    # Columns to store wins and losses
    season_df['homeWins'] = 0
    season_df['homeLosses'] = 0
    season_df['awayWins'] = 0
    season_df['awayLosses'] = 0

    # Iterate over each game in the season
    for idx, row in season_df.iterrows():
        home_team = row['homeTeam']
        away_team = row['awayTeam']
        winner = row['winner']

        # Initialize win/loss counts for teams if not already present
        if home_team not in win_counts:
            win_counts[home_team] = 0
            loss_counts[home_team] = 0
        if away_team not in win_counts:
            win_counts[away_team] = 0
            loss_counts[away_team] = 0

        # Update win/loss counts based on the winner
        if winner == home_team:
            win_counts[home_team] += 1
            loss_counts[away_team] += 1
        elif winner == away_team:
            win_counts[away_team] += 1
            loss_counts[home_team] += 1

        # Assign current win/loss counts to the DataFrame
        season_df.at[idx, 'homeWins'] = win_counts[home_team]
        season_df.at[idx, 'homeLosses'] = loss_counts[home_team]
        season_df.at[idx, 'awayWins'] = win_counts[away_team]
        season_df.at[idx, 'awayLosses'] = loss_counts[away_team]

    return season_df

# Apply the win/loss calculation for each season
dfMine = dfMine.groupby('season', group_keys=False).apply(calculate_win_loss)
dfMine

,date,homeTeam,homeTeam_id,homeTeam_points_total,homeTeam_points_q1,homeTeam_points_q2,homeTeam_points_q3,homeTeam_points_q4,homeTeam_points_1OT,homeTeam_points_2OT,...,winner,season,homeWinPct,awayWinPct,gameNumber,isPlayoffGame,homeWins,homeLosses,awayWins,awayLosses
0,2007-04-18,Utah Jazz,UTA,101,20,30,25,26,0,0,...,Utah Jazz,2006-2007,100.000000,0.000000,1,False,1,0,0,1
1,2007-04-18,Toronto Raptors,TOR,119,28,29,29,33,0,0,...,Philadelphia 76ers,2006-2007,0.000000,100.000000,2,False,0,1,1,0
2,2007-04-18,Seattle Supersonics,SEA,75,16,21,22,16,0,0,...,Dallas Mavericks,2006-2007,0.000000,100.000000,3,False,0,1,1,0
3,2007-04-18,San Antonio Spurs,SAS,77,16,24,22,15,0,0,...,Denver Nuggets,2006-2007,0.000000,100.000000,4,False,0,1,1,0
4,2007-04-18,Sacramento Kings,SAC,106,30,27,23,26,0,0,...,Los Angeles Lakers,2006-2007,0.000000,100.000000,5,False,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22248,2024-12-13,Minnesota Timberwolves,MIN,97,32,18,27,20,0,0,...,Minnesota Timberwolves,2024-2025,54.166667,52.000000,372,False,13,11,13,12
22249,2024-12-13,Philadelphia 76ers,PHI,107,22,28,33,24,0,0,...,Indiana Pacers,2024-2025,30.434783,42.307692,373,False,7,16,11,15
22250,2024-12-13,Cleveland Cavaliers,CLE,115,25,32,30,28,0,0,...,Cleveland Cavaliers,2024-2025,84.615385,13.043478,374,False,22,4,3,20
22251,2024-12-14,Oklahoma City Thunder,OKC,111,18,23,34,36,0,0,...,Oklahoma City Thunder,2024-2025,80.000000,65.384615,375,False,20,5,17,9


In [49]:
subjectID_dict = {
    "ATL": 1,
    "BOS": 2,
    "BKN": 3,
    "CHA": 4,
    "CHI": 5,
    "CLE": 6,
    "DAL": 7,
    "DEN": 8,
    "DET": 9,
    "GSW": 10,
    "HOU": 11,
    "IND": 12,
    "LAC": 13,
    "LAL": 14,
    "MEM": 15,
    "MIA": 16,
    "MIL": 17,
    "MIN": 18,
    "NOP": 19,
    "NYK": 20,
    "OKC": 21,
    "ORL": 22,
    "PHI": 23,
    "PHX": 24,
    "POR": 25,
    "SAC": 26,
    "SAS": 27,
    "TOR": 28,
    "UTA": 29,
    "WAS": 30,
    "NOH": 19,
    "NJN": 3,
    "SEA": 21
}

In [51]:
def df_to_X_y(df, dfWindow, window_size):
    X = []
    y = []
    #each team must have played window_size games before data can be extracted
    for index, row in df.iterrows():
        homeTeam_id = row['homeTeamSubject_id']
        awayTeam_id = row['awayTeamSubject_id']
        date = row['date']
        #limit dfWindow to include rows that occured before date of row we are on
        dfWindow_before_date = dfWindow[dfWindow['date']< date]
        homeTeam_occurrences = dfWindow_before_date[dfWindow_before_date['team_id'] == homeTeam_id].shape[0]
        awayTeam_occurrences = dfWindow_before_date[dfWindow_before_date['team_id'] == awayTeam_id].shape[0]
        if homeTeam_occurrences > window_size and awayTeam_occurrences > window_size:
            homeTeam_window = (dfWindow_before_date[dfWindow_before_date['team_id'] == homeTeam_id].sort_values(by='date', ascending=False).head(window_size).drop(columns=['date', 'team_id']))
            awayTeam_window = (dfWindow_before_date[dfWindow_before_date['team_id'] == awayTeam_id].sort_values(by='date', ascending=False).head(window_size).drop(columns=['date', 'team_id']))
            homeTeam_window = homeTeam_window.to_numpy()
            awayTeam_window = awayTeam_window.to_numpy()
            combined_window = np.hstack((homeTeam_window, awayTeam_window))
            X.append(combined_window)
            y.append(row['winner_binary'])
    
    return np.array(X), np.array(y)   


In [56]:
df = dfMine
df

,date,homeTeam,homeTeam_id,homeTeam_points_total,homeTeam_points_q1,homeTeam_points_q2,homeTeam_points_q3,homeTeam_points_q4,homeTeam_points_1OT,homeTeam_points_2OT,...,winner,season,homeWinPct,awayWinPct,gameNumber,isPlayoffGame,homeWins,homeLosses,awayWins,awayLosses
0,2007-04-18,Utah Jazz,UTA,101,20,30,25,26,0,0,...,Utah Jazz,2006-2007,100.000000,0.000000,1,False,1,0,0,1
1,2007-04-18,Toronto Raptors,TOR,119,28,29,29,33,0,0,...,Philadelphia 76ers,2006-2007,0.000000,100.000000,2,False,0,1,1,0
2,2007-04-18,Seattle Supersonics,SEA,75,16,21,22,16,0,0,...,Dallas Mavericks,2006-2007,0.000000,100.000000,3,False,0,1,1,0
3,2007-04-18,San Antonio Spurs,SAS,77,16,24,22,15,0,0,...,Denver Nuggets,2006-2007,0.000000,100.000000,4,False,0,1,1,0
4,2007-04-18,Sacramento Kings,SAC,106,30,27,23,26,0,0,...,Los Angeles Lakers,2006-2007,0.000000,100.000000,5,False,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22248,2024-12-13,Minnesota Timberwolves,MIN,97,32,18,27,20,0,0,...,Minnesota Timberwolves,2024-2025,54.166667,52.000000,372,False,13,11,13,12
22249,2024-12-13,Philadelphia 76ers,PHI,107,22,28,33,24,0,0,...,Indiana Pacers,2024-2025,30.434783,42.307692,373,False,7,16,11,15
22250,2024-12-13,Cleveland Cavaliers,CLE,115,25,32,30,28,0,0,...,Cleveland Cavaliers,2024-2025,84.615385,13.043478,374,False,22,4,3,20
22251,2024-12-14,Oklahoma City Thunder,OKC,111,18,23,34,36,0,0,...,Oklahoma City Thunder,2024-2025,80.000000,65.384615,375,False,20,5,17,9


In [57]:
df["homeTeamSubject_id"] = df["homeTeam_id"].map(subjectID_dict)
df["awayTeamSubject_id"] = df["awayTeam_id"].map(subjectID_dict)

cols = df.columns.tolist()
cols.insert(cols.index("homeTeam_id") + 1, cols.pop(cols.index("homeTeamSubject_id")))
cols.insert(cols.index("awayTeam_id") + 1, cols.pop(cols.index("awayTeamSubject_id")))
df = df[cols]

df

,date,homeTeam,homeTeam_id,homeTeamSubject_id,homeTeam_points_total,homeTeam_points_q1,homeTeam_points_q2,homeTeam_points_q3,homeTeam_points_q4,homeTeam_points_1OT,...,winner,season,homeWinPct,awayWinPct,gameNumber,isPlayoffGame,homeWins,homeLosses,awayWins,awayLosses
0,2007-04-18,Utah Jazz,UTA,29,101,20,30,25,26,0,...,Utah Jazz,2006-2007,100.000000,0.000000,1,False,1,0,0,1
1,2007-04-18,Toronto Raptors,TOR,28,119,28,29,29,33,0,...,Philadelphia 76ers,2006-2007,0.000000,100.000000,2,False,0,1,1,0
2,2007-04-18,Seattle Supersonics,SEA,21,75,16,21,22,16,0,...,Dallas Mavericks,2006-2007,0.000000,100.000000,3,False,0,1,1,0
3,2007-04-18,San Antonio Spurs,SAS,27,77,16,24,22,15,0,...,Denver Nuggets,2006-2007,0.000000,100.000000,4,False,0,1,1,0
4,2007-04-18,Sacramento Kings,SAC,26,106,30,27,23,26,0,...,Los Angeles Lakers,2006-2007,0.000000,100.000000,5,False,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22248,2024-12-13,Minnesota Timberwolves,MIN,18,97,32,18,27,20,0,...,Minnesota Timberwolves,2024-2025,54.166667,52.000000,372,False,13,11,13,12
22249,2024-12-13,Philadelphia 76ers,PHI,23,107,22,28,33,24,0,...,Indiana Pacers,2024-2025,30.434783,42.307692,373,False,7,16,11,15
22250,2024-12-13,Cleveland Cavaliers,CLE,6,115,25,32,30,28,0,...,Cleveland Cavaliers,2024-2025,84.615385,13.043478,374,False,22,4,3,20
22251,2024-12-14,Oklahoma City Thunder,OKC,21,111,18,23,34,36,0,...,Oklahoma City Thunder,2024-2025,80.000000,65.384615,375,False,20,5,17,9


In [74]:
print(df.columns)

Index(['date', 'homeTeam', 'homeTeam_id', 'homeTeamSubject_id',
       'homeTeam_points_total', 'homeTeam_points_q1', 'homeTeam_points_q2',
       'homeTeam_points_q3', 'homeTeam_points_q4', 'homeTeam_points_1OT',
       'homeTeam_points_2OT', 'homeTeam_points_3OT', 'homeTeam_points_4OT',
       'homeTeam_FG', 'homeTeam_FG_made', 'homeTeam_3P', 'homeTeam_3P_made',
       'homeTeam_O_Reb', 'homeTeam_D_Reb', 'homeTeam_Total_Reb',
       'homeTeam_Ast', 'homeTeam_Stl', 'homeTeam_Blk', 'homeTeam_TO',
       'homeTeam_PF', 'awayTeam', 'awayTeam_id', 'awayTeamSubject_id',
       'awayTeam_points_total', 'awayTeam_points_q1', 'awayTeam_points_q2',
       'awayTeam_points_q3', 'awayTeam_points_q4', 'awayTeam_points_1OT',
       'awayTeam_points_2OT', 'awayTeam_points_3OT', 'awayTeam_points_4OT',
       'awayTeam_FG', 'awayTeam_FG_made', 'awayTeam_3P', 'awayTeam_3P_made',
       'awayTeam_O_Reb', 'awayTeam_D_Reb', 'awayTeam_Total_Reb',
       'awayTeam_Ast', 'awayTeam_Stl', 'awayTeam_Blk', 'awa

In [75]:
dfLim = df[['date', 'homeTeam', 'homeTeam_id', 'homeTeamSubject_id',
       'homeTeam_points_total', 'homeTeam_points_q1', 'homeTeam_points_q2',
       'homeTeam_points_q3', 'homeTeam_points_q4',
       'homeTeam_FG', 'homeTeam_FG_made', 'homeTeam_3P', 'homeTeam_3P_made',
       'homeTeam_O_Reb', 'homeTeam_D_Reb', 'homeTeam_Total_Reb',
       'homeTeam_Ast', 'homeTeam_Stl', 'homeTeam_Blk', 'homeTeam_TO',
       'homeTeam_PF', 'awayTeam', 'awayTeam_id', 'awayTeamSubject_id',
       'awayTeam_points_total', 'awayTeam_points_q1', 'awayTeam_points_q2',
       'awayTeam_points_q3', 'awayTeam_points_q4',
       'awayTeam_FG', 'awayTeam_FG_made', 'awayTeam_3P', 'awayTeam_3P_made',
       'awayTeam_O_Reb', 'awayTeam_D_Reb', 'awayTeam_Total_Reb',
       'awayTeam_Ast', 'awayTeam_Stl', 'awayTeam_Blk', 'awayTeam_TO',
       'awayTeam_PF', 'homeWinPct', 'awayWinPct', 
       'homeWins', 'homeLosses', 'awayWins',
       'awayLosses', 'winner_binary']]
dfLim

,date,homeTeam,homeTeam_id,homeTeamSubject_id,homeTeam_points_total,homeTeam_points_q1,homeTeam_points_q2,homeTeam_points_q3,homeTeam_points_q4,homeTeam_FG,...,awayTeam_Blk,awayTeam_TO,awayTeam_PF,homeWinPct,awayWinPct,homeWins,homeLosses,awayWins,awayLosses,winner_binary
0,2007-04-18,Utah Jazz,UTA,29,101,20,30,25,26,36-73,...,2,11,29,100.000000,0.000000,1,0,0,1,0
1,2007-04-18,Toronto Raptors,TOR,28,119,28,29,29,33,46-86,...,5,15,25,0.000000,100.000000,0,1,1,0,1
2,2007-04-18,Seattle Supersonics,SEA,21,75,16,21,22,16,29-69,...,5,9,21,0.000000,100.000000,0,1,1,0,1
3,2007-04-18,San Antonio Spurs,SAS,27,77,16,24,22,15,29-81,...,3,15,21,0.000000,100.000000,0,1,1,0,1
4,2007-04-18,Sacramento Kings,SAC,26,106,30,27,23,26,40-83,...,6,14,26,0.000000,100.000000,0,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22248,2024-12-13,Minnesota Timberwolves,MIN,18,97,32,18,27,20,35-87,...,6,21,21,54.166667,52.000000,13,11,13,12,0
22249,2024-12-13,Philadelphia 76ers,PHI,23,107,22,28,33,24,37-85,...,4,10,21,30.434783,42.307692,7,16,11,15,1
22250,2024-12-13,Cleveland Cavaliers,CLE,6,115,25,32,30,28,36-91,...,7,17,28,84.615385,13.043478,22,4,3,20,0
22251,2024-12-14,Oklahoma City Thunder,OKC,21,111,18,23,34,36,36-80,...,3,16,25,80.000000,65.384615,20,5,17,9,0


In [72]:
dfLim.insert(dfLim.columns.get_loc('homeTeam_3P') + 1, 'homeTeam_3P_made', dfLim['homeTeam_3P'].str.split('-').apply(lambda x: int(x[0]) if isinstance(x, list) and len(x) == 2 else np.nan))
dfLim.insert(dfLim.columns.get_loc('awayTeam_3P') + 1, 'awayTeam_3P_made', dfLim['awayTeam_3P'].str.split('-').apply(lambda x: int(x[0]) if isinstance(x, list) and len(x) == 2 else np.nan))

#adding FG made
dfLim.insert(dfLim.columns.get_loc('homeTeam_FG') + 1, 'homeTeam_FG_made', dfLim['homeTeam_FG'].str.split('-').apply(lambda x: int(x[0]) if isinstance(x, list) and len(x) == 2 else np.nan))
dfLim.insert(dfLim.columns.get_loc('awayTeam_FG') + 1, 'awayTeam_FG_made', dfLim['awayTeam_FG'].str.split('-').apply(lambda x: int(x[0]) if isinstance(x, list) and len(x) == 2 else np.nan))

In [73]:
dfLim['winner_binary'] = (dfLim['winner'] == df['awayTeam']).astype(int)
dfLim

C:\Users\Max Braithwaite\AppData\Local\Temp\ipykernel_7820\472979383.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfLim['winner_binary'] = (dfLim['winner'] == df['awayTeam']).astype(int)


,date,homeTeam,homeTeam_id,homeTeamSubject_id,homeTeam_points_total,homeTeam_points_q1,homeTeam_points_q2,homeTeam_points_q3,homeTeam_points_q4,homeTeam_points_1OT,...,season,homeWinPct,awayWinPct,gameNumber,isPlayoffGame,homeWins,homeLosses,awayWins,awayLosses,winner_binary
0,2007-04-18,Utah Jazz,UTA,29,101,20,30,25,26,0,...,2006-2007,100.000000,0.000000,1,False,1,0,0,1,0
1,2007-04-18,Toronto Raptors,TOR,28,119,28,29,29,33,0,...,2006-2007,0.000000,100.000000,2,False,0,1,1,0,1
2,2007-04-18,Seattle Supersonics,SEA,21,75,16,21,22,16,0,...,2006-2007,0.000000,100.000000,3,False,0,1,1,0,1
3,2007-04-18,San Antonio Spurs,SAS,27,77,16,24,22,15,0,...,2006-2007,0.000000,100.000000,4,False,0,1,1,0,1
4,2007-04-18,Sacramento Kings,SAC,26,106,30,27,23,26,0,...,2006-2007,0.000000,100.000000,5,False,0,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22248,2024-12-13,Minnesota Timberwolves,MIN,18,97,32,18,27,20,0,...,2024-2025,54.166667,52.000000,372,False,13,11,13,12,0
22249,2024-12-13,Philadelphia 76ers,PHI,23,107,22,28,33,24,0,...,2024-2025,30.434783,42.307692,373,False,7,16,11,15,1
22250,2024-12-13,Cleveland Cavaliers,CLE,6,115,25,32,30,28,0,...,2024-2025,84.615385,13.043478,374,False,22,4,3,20,0
22251,2024-12-14,Oklahoma City Thunder,OKC,21,111,18,23,34,36,0,...,2024-2025,80.000000,65.384615,375,False,20,5,17,9,0


In [64]:
#Format as percentage
dfLim['homeTeam_3P'] = dfLim['homeTeam_3P'].str.split('-').apply(lambda x: int(x[0]) / int(x[1]) if isinstance(x, list) and len(x) == 2 else np.nan)
dfLim['awayTeam_3P'] = dfLim['awayTeam_3P'].str.split('-').apply(lambda x: int(x[0]) / int(x[1]) if isinstance(x, list) and len(x) == 2 else np.nan)
dfLim['homeTeam_FG'] = dfLim['homeTeam_FG'].str.split('-').apply(lambda x: int(x[0]) / int(x[1]) if isinstance(x, list) and len(x) == 2 else np.nan)
dfLim['awayTeam_FG'] = dfLim['awayTeam_FG'].str.split('-').apply(lambda x: int(x[0]) / int(x[1]) if isinstance(x, list) and len(x) == 2 else np.nan)

dfLim

C:\Users\Max Braithwaite\AppData\Local\Temp\ipykernel_7820\3179912699.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfLim['homeTeam_3P'] = dfLim['homeTeam_3P'].str.split('-').apply(lambda x: int(x[0]) / int(x[1]) if isinstance(x, list) and len(x) == 2 else np.nan)
C:\Users\Max Braithwaite\AppData\Local\Temp\ipykernel_7820\3179912699.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfLim['awayTeam_3P'] = dfLim['awayTeam_3P'].str.split('-').apply(lambda x: int(x[0]) / int(x[1]) if isinstance(x, list)

,date,homeTeam,homeTeamSubject_id,homeTeam_points_total,homeTeam_Stl,homeTeam_Blk,homeTeam_points_q1,homeTeam_points_q2,homeTeam_points_q3,homeTeam_points_q4,...,awayTeam_points_q4,awayTeam_3P,awayTeam_3P_made,awayTeam_FG,awayTeam_FG_made,awayTeam_Total_Reb,awayTeam_Ast,awayWinPct,winner,winner_binary
0,2007-04-18,Utah Jazz,29,101,5,0,20,30,25,26,...,20,0.240000,6,0.418919,31,25,17,0.000000,Utah Jazz,0
1,2007-04-18,Toronto Raptors,28,119,8,3,28,29,29,33,...,36,0.555556,5,0.530120,44,41,22,100.000000,Philadelphia 76ers,1
2,2007-04-18,Seattle Supersonics,21,75,5,3,16,21,22,16,...,17,0.307692,4,0.530864,43,38,26,100.000000,Dallas Mavericks,1
3,2007-04-18,San Antonio Spurs,27,77,7,3,16,24,22,15,...,21,0.423077,11,0.494118,42,55,28,100.000000,Denver Nuggets,1
4,2007-04-18,Sacramento Kings,26,106,5,3,30,27,23,26,...,29,0.142857,2,0.569620,45,47,19,100.000000,Los Angeles Lakers,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22248,2024-12-13,Minnesota Timberwolves,18,97,12,5,32,18,27,20,...,21,0.285714,10,0.383721,33,53,17,52.000000,Minnesota Timberwolves,0
22249,2024-12-13,Philadelphia 76ers,23,107,5,4,22,28,33,24,...,31,0.368421,14,0.516484,47,41,35,42.307692,Indiana Pacers,1
22250,2024-12-13,Cleveland Cavaliers,6,115,10,5,25,32,30,28,...,25,0.350000,14,0.476744,41,44,24,13.043478,Cleveland Cavaliers,0
22251,2024-12-14,Oklahoma City Thunder,21,111,12,4,18,23,34,36,...,27,0.239130,11,0.364583,35,49,17,65.384615,Oklahoma City Thunder,0


In [69]:
dfIDtoStat = pd.DataFrame(columns=['date', 'team_id', 'team_points_total', '3P%', '3P_made', 'FG%', 'FG_made', 'Total_Reb', 'O_Reb', 'D_Reb', 'team_Ast', 'team_Stl', 'team_Blk', 'team_TO', 'team_PF' 'team_points_q1', 'team_points_q2', 'team_points_q3', 'team_points_q4', 'winPct'])
dfIDtoStat

,date,team_id,team_points_total,3P%,3P_made,FG%,FG_made,Total_Reb,O_Reb,D_Reb,team_Ast,team_Stl,team_Blk,team_TO,team_PFteam_points_q1,team_points_q2,team_points_q3,team_points_q4,winPct


In [ ]:
#populate dfIDtoStat by interlacing performances
for index, row in dfLim.iterrows():
    date = row['date']
    homeTeam_id = row['homeTeamSubject_id']
    homeTeam_points_total = row['homeTeam_points_total']
    homeTeam_3P = row['homeTeam_3P']
    homeTeam_3P_made = row['homeTeam_3P_made']
    homeTeam_FG = row['homeTeam_FG']
    homeTeam_FG_made = row['homeTeam_FG_made']
    homeTeam_Total_Reb = row['homeTeam_Total_Reb']
    homeTeam_O_Reb = row['homeTeam_O_Reb']
    homeTeam_D_Reb = row['homeTeam_D_Reb']
    homeTeam_Ast = row['homeTeam_Ast']
    homeTeam_Stl = row['homeTeam_Stl']
    homeTeam_Blk = rpw
    awayTeam_id = row['awayTeamSubject_id']
    awayTeam_points_total = row['awayTeam_points_total']
    awayTeam_3P = row['homeTeam_3P']
    awayTeam_FG = row['awayTeam_FG']
    awayTeam_FG_made = row['awayTeam_FG_made']
    #added these new variables into this
    dfIDtoStat.loc[len(dfIDtoStat)] = {"date": date, 'team_id': homeTeam_id, 'team_points_total':homeTeam_points_total, '3P%':homeTeam_3P, '3P_made':homeTeam_3P_made, 'FG%':homeTeam_FG, 'FG_made':homeTeam_FG_made}
    dfIDtoStat.loc[len(dfIDtoStat)] = {"date": date, 'team_id': awayTeam_id, 'team_points_total':awayTeam_points_total, '3P%':awayTeam_3P, '3P_made':awayTeam_3P_made, 'FG%':awayTeam_FG, 'FG_made':awayTeam_FG_made}
dfIDtoStat